# Usage

The basic usage of cmomy is to manipulate central moments.  
We measure two quantities pretend quantities.  The 'energy' and the 'position' of a thing.  We'll construct the average value for each record. Lets say 100 samples each.  

In [1]:
import numpy as np
import xarray as xr
import cmomy 

nsamp = 100
energy = xr.DataArray(np.random.rand(nsamp), dims=['samp'])

position = xr.DataArray(np.random.rand(nsamp, 3), dims=['samp','dim'])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nsamp), dims=['samp'])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim='samp', mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim='samp', mom=3)

In [2]:
ce

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([5.01601826e+01, 4.81659442e-01, 9.75464611e-02, 3.00281423e-03])
Dimensions without coordinates: mom_0

In [3]:
cp

<xCentralMoments(val_shape=(3,), mom=(3,))>
<xarray.DataArray (dim: 3, mom_0: 4)>
array([[ 5.01601826e+01,  5.34301885e-01,  7.99008214e-02,
        -1.07095037e-03],
       [ 5.01601826e+01,  5.41399842e-01,  7.36411728e-02,
        -8.76362073e-05],
       [ 5.01601826e+01,  4.16773751e-01,  7.35260399e-02,
         9.34992671e-03]])
Dimensions without coordinates: dim, mom_0

## Basic attribues

Notice that there are three `shape` parameters associated with a `CentraMoments` object:
* `mom_shape` : shape of the moments.  For single variable, tuple (mom+1,).  For comoments, (mom_0+1, mom_1+1)
* `val_shape`: shape of the 'values' part of the data.  For scalar data, `val_shape = ()`.  For vectory data, 
* `shape`: total shape of wrapped moments data.

In [4]:
ce.mom_shape, ce.val_shape, ce.shape

((4,), (), (4,))

In [5]:
cp.mom_shape, cp.val_shape, cp.shape

((4,), (3,), (3, 4))

To access the underlying data, use the `values` attribute.  `values[...,0]`  is the total weight, `values[...,1]` is the average, and `values[...,k>1]` is the 'kth' central moment.

In [6]:
ce.values

<xarray.DataArray (mom_0: 4)>
array([5.01601826e+01, 4.81659442e-01, 9.75464611e-02, 3.00281423e-03])
Dimensions without coordinates: mom_0

To access all the central moments (zeroth and first included), use

In [7]:
ce.cmom()

<xarray.DataArray (mom_0: 4)>
array([1.        , 0.        , 0.09754646, 0.00300281])
Dimensions without coordinates: mom_0

Likewise, the central moments can be converted to raw moments. 

In [8]:
#<x**k>
ce.rmom()

<xarray.DataArray (mom_0: 4)>
array([1.        , 0.48165944, 0.32954228, 0.25569831])
Dimensions without coordinates: mom_0

Additionally, there are `DataArray` like attributes

In [9]:
ce.coords

Coordinates:
    *empty*

In [10]:
ce.attrs

{}

In [11]:
ce.sizes

Frozen({'mom_0': 4})

In [12]:
ce.dims

('mom_0',)

## Manipulating momements

So we have our averages.  Cool.  Not very special.  But what if instead we repeat our experiment.  Let's say we did the experiment 10 times each time giving a single record.   Then our data would look like

In [13]:
nsamp = 100
nrec = 10
energy = xr.DataArray(np.random.rand(nrec, nsamp), dims=['rec','samp'])
position = xr.DataArray(np.random.rand(nrec, nsamp, 3), dims=['rec','samp','dim'])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nrec, nsamp), dims=['rec','samp'])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim='samp', mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim='samp', mom=3)

Consider just the energy.  We suspect that there is some correlation between the experiments (they where done in rapid succession).  So we'd like to consider two adjacent experiments as one experiment.  For this, we can use the `block` method.

In [14]:
ce

<xCentralMoments(val_shape=(10,), mom=(3,))>
<xarray.DataArray (rec: 10, mom_0: 4)>
array([[ 4.90145030e+01,  4.67774632e-01,  8.60043171e-02,
         4.67373486e-03],
       [ 5.10808094e+01,  5.10850014e-01,  8.49355693e-02,
        -1.71555081e-03],
       [ 5.30056534e+01,  5.13824464e-01,  6.44206768e-02,
         1.36908018e-03],
       [ 5.34498667e+01,  4.82672363e-01,  7.21531752e-02,
         1.68504405e-04],
       [ 4.69453882e+01,  5.07512609e-01,  8.96390950e-02,
         2.10326835e-03],
       [ 4.95022887e+01,  4.52467137e-01,  8.42672774e-02,
         2.54091302e-03],
       [ 4.67317964e+01,  4.73904166e-01,  8.13592319e-02,
         8.31723649e-03],
       [ 5.12223347e+01,  5.31301178e-01,  7.07136888e-02,
        -3.30823499e-03],
       [ 5.37551546e+01,  4.72692027e-01,  7.23553470e-02,
         7.40299840e-05],
       [ 4.55623338e+01,  5.46014887e-01,  7.08405499e-02,
        -6.92438655e-03]])
Dimensions without coordinates: rec, mom_0

In [15]:
ce.block(block_size=2, dim='rec')

<xCentralMoments(val_shape=(5,), mom=(3,))>
<xarray.DataArray (rec: 5, mom_0: 4)>
array([[ 1.00095312e+02,  4.89756934e-01,  8.59225864e-02,
         1.37821864e-03],
       [ 1.06455520e+02,  4.98183418e-01,  6.85456680e-02,
         5.85659440e-04],
       [ 9.64476770e+01,  4.79260225e-01,  8.76389494e-02,
         2.55061128e-03],
       [ 9.79541311e+01,  5.03918306e-01,  7.66143204e-02,
         1.77855965e-03],
       [ 9.93174884e+01,  5.06329211e-01,  7.29953416e-02,
        -3.21118519e-03]])
Dimensions without coordinates: rec, mom_0

Instead, we can resample the already averaged data.  We produce a 20 new samples taken from the original (averaged) data.

In [16]:
ce.resample_and_reduce(nrep=20, dim='rec')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 4.93150624e+02,  4.94227305e-01,  7.74323119e-02,
         1.94477940e-03],
       [ 5.04787611e+02,  5.15693111e-01,  7.25708383e-02,
        -1.04687858e-04],
       [ 4.91590693e+02,  5.03264907e-01,  8.25692115e-02,
        -5.26125180e-04],
       [ 5.05999660e+02,  4.89626422e-01,  7.88500359e-02,
         1.19800890e-03],
       [ 4.91520127e+02,  5.01422758e-01,  7.85118174e-02,
         3.82752360e-04],
       [ 4.74064581e+02,  4.95574276e-01,  8.64898417e-02,
         2.52467109e-03],
       [ 4.98113493e+02,  5.02755257e-01,  7.61912295e-02,
         6.78815741e-05],
       [ 4.86883417e+02,  4.78844156e-01,  8.52906666e-02,
         3.69841064e-03],
       [ 5.02337932e+02,  5.06506085e-01,  7.68584049e-02,
         1.18329348e-03],
       [ 5.08596530e+02,  4.88410054e-01,  7.76762168e-02,
         1.16189048e-03],
       [ 4.84252045e+02,  5.05977593e-01,  7.69670041e-02,
       

This is differnt than the usual 'resample values'.  This is also available if the original data is available.

In [17]:
a = energy.stack(c=['rec','samp'])

In [18]:
b = w.stack(c=['rec','samp'])

In [19]:
# consider 'all' the data for this
out = cmomy.xCentralMoments.from_resample_vals(energy.stack(c=['rec','samp']), w=w.stack(c=['rec','samp']), dim='c', nrep=20, mom=3)

We can also reduce our original data across all the records

In [20]:
ce.reduce(dim='rec')

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([5.00270129e+02, 4.95589272e-01, 7.82628743e-02, 4.29121941e-04])
Dimensions without coordinates: mom_0